In [ ]:
import paho.mqtt.client as mqtt
import time
import random

# MQTT settings
BROKER = "158.39.162.129" 
PORT = 1883
ROLE = "follower"
LEADER_IP = "192.168.1.12"

if ROLE == "follower":
    print(f"This JetBot is a Follower. Connecting to Leader at IP: {LEADER_IP}")
else:
    print("Error: Role not defined properly")
    exit()

TOPIC_LEADER = "platoon/leader"
TOPIC_FOLLOWERS = "platoon/followers"
TOPIC_STATUS = "platoon/status"
TOPIC_SYNC = "platoon/sync"

last_message = ""

# Initialize the MQTT client
client = mqtt.Client()

def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to broker successfully.")
        client.subscribe([(TOPIC_LEADER, 0), (TOPIC_SYNC, 0)])  # Subscribing to multiple topics
    else:
        print(f"Connection failed with code {rc}")

def on_disconnect(client, userdata, rc):
    if rc != 0:
        print("Unexpected disconnection. Attempting to reconnect...")
    else:
        print("Disconnected cleanly.")

def on_message(client, userdata, msg):
    global last_message
    if msg.topic == TOPIC_LEADER:
        command = msg.payload.decode()
        last_message = command
        print(f"Received command from leader: {command}")
        if command == "start":
            print("Moving forward...")
        elif command == "stop":
            print("Stopping...")
        elif command == "move right":
            print("Moving right...")
        elif command == "move left":
            print("Moving left...")
        elif command == "speed up":
            print("Speeding up...")
        elif command == "slow down":
            print("Slowing down...")
        else:
            print("Unknown command.")
        client.publish(TOPIC_FOLLOWERS, f"Acknowledged: {command}")

    elif msg.topic == TOPIC_SYNC:
        sync_signal = msg.payload.decode()
        print(f"Received sync signal: {sync_signal}")
        if sync_signal == "sync":
            print("Synchronizing...")
        elif sync_signal == "align":
            print("Aligning position...")
        else:
            print("Unknown sync signal.")
            
def get_last_message():
    return last_message

# Attach callbacks
client.on_connect = on_connect
client.on_disconnect = on_disconnect
client.on_message = on_message

# Connect to the broker
try:
    client.connect(BROKER, PORT, keepalive=60)
except Exception as e:
    print(f"Error connecting to broker: {e}")
    exit()

# Start the MQTT client loop
client.loop_start()
